# 서론 : EDA를 시작하기에 앞서서 
본 프로젝트의 목적은 EDA를 통해 거래 데이터를 분석 파악한 뒤, 적절한 데이터 전처리를 통해 데이터에서 유의미한 인사이트를 얻을 수 있도록 가공하고, 머신러닝 모델 학습을 통해 unseen data의 거래 사기 예측을 효과적으로 수행하는 것이다.

해당 데이터셋은 개우 많은 column, 즉 차원을 지니고 있으므로 적절한 판단을 통해 결측치를 처리하고 분석에 필요 없는 column을 버려야 한다.
따라서 EDA의 목적은 위에 기재한 것으로 둔다.

# 라이브러리 임포트

In [ ]:
!pip install -U vega_datasets notebook vega

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff

# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from xgboost import XGBClassifier
import xgboost as xgb

## Hyperopt modules
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics
import time
import lightgbm as lgb

import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import eli5
import shap
from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

alt.renderers.enable('notebook')

%env JOBLIB_TEMP_FOLDER=/tmp

import json
from numba import jit


from tqdm import tqdm_notebook

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics

from itertools import product

import altair as alt
from IPython.display import HTML


import os
import gc
print(os.listdir("../input"))

# 데이터 임포트

In [ ]:
folder_path = '../input/ieee-fraud-detection/'
df_id = pd.read_csv(f'{folder_path}/train_identity.csv')
df_trans = pd.read_csv(f'{folder_path}train_transaction.csv')
train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
test_transaction = pd.read_csv(f'{folder_path}test_transaction.csv' )
test_identity = pd.read_csv(f'{folder_path}test_identity.csv')

train = pd.merge(df_trans, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [ ]:
sub = pd.read_csv(f'{folder_path}sample_submission.csv')


# 함수 해석
def resumetable(df):
* 데이터셋의 Shape를 확인하고, 데이터 type를 확인하며, 인덱스를 재정렬하고, 결측치 Null값의 개수를 확인하고, unique한 value의 갯수를 확인하고, 첫번째에서부터 세번째까지의 value를 확인한다. 이렇게 우리는 개괄적인 데이터셋의 형태를 확인할 수 있다. 


def reduce_mem_usage(df, verbose=True):
* verbose=True 는 함수 수행시 발생하는 상세한 정보들을 표준 출력으로 자세히 내보낼 것인가를 나타낸다. 이 함수의 for col in df.columns: 를 통해 df[col]의 min 과 max를 확인하고, 적절한 대소 비교를 통해 데이터타입을 변환한다.

def CalcOutliers(df_num): 
* 아웃라이어를 계산하는 함수를 만든다. array를 전달받아 평균값과 표준편차 값을 확인하고, 그것을 upper과 lower로 나누어 각각 계산한다. cut line의 값을 임의로 설정할 수도 있지만, 일단은 3으로 둔다.

In [ ]:
# datatype shape 찍는 함수
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary


## Dataframe size 줄이는 함수
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


### 아웃라이어 관리 함수
def CalcOutliers(df_num): 

    # array의 평균 및 표준 계산
    data_mean, data_std = np.mean(df_num), np.std(df_num)

    # cut line 값을 설정한다.
    # 이 value값을 임의로 변경할 수도 있다.
    cut = data_std * 3

    #더 높은 cut 및 더 낮은 cut 계산
    lower, upper = data_mean - cut, data_mean + cut

    # lower, higher and total 아웃라이어 값이 담긴 array를 만든다. 
    outliers_lower = [x for x in df_num if x < lower]
    outliers_higher = [x for x in df_num if x > upper]
    outliers_total = [x for x in df_num if x < lower or x > upper]

    # 아웃라이어를 제외한 array
    outliers_removed = [x for x in df_num if x > lower and x < upper]
    
    print('Identified lowest outliers: %d' % len(outliers_lower)) 
    print('Identified upper outliers: %d' % len(outliers_higher)) 
    print('Total outlier observations: %d' % len(outliers_total)) 
    print('Non-outlier observations: %d' % len(outliers_removed)) 
    print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4))
    
    return

앞서 만든 함수를 활용해 데이터셋의 메모리를 축소한다.

In [ ]:
## 메모리 축소
df_trans = reduce_mem_usage(df_trans)
df_id = reduce_mem_usage(df_id)
test_transaction = reduce_mem_usage(test_transaction)
test_identity = reduce_mem_usage(test_identity)

In [ ]:
resumetable(df_trans)[:25]

## 위를 통해 얻을 수 있는 인사이트 
resumetable 앞서 만든 함수를 통해 데이터의 개괄을 확인해보았다.
어떤 인사이트를 가지고 알 수 있는지 가볍게 정리해보자.

* resumetable 함수를 통해,  column의 데이터타입에 object type이 섞여 있다는 것을 알 수 있다. 
* 또한 missing 항목을 보면, TransactionID, isFraud, TransactionDT, TransactionAmt, ProductCD, card1, 마스킹된 Cn 피쳐들에 결측치가 없다는 것도 알 수 있다.
* TransactionID의 모든 value는 전부 unique하다는 것을 알 수 있다.
* ProductCD의 경우, 데이터가 무엇을 뜻하는지 알기 어렵다는 것, 즉 실효성 있는 데이터가 아니라는 추측을 할 수 있다.
* P_emaildomain과 R_emaildomain의 경우, 적절한 Value값 통일이 필요하다는 것을 알 수 있다.
* 하지만 column의 수가 매우 많으므로, 이런 방식으로 모든 피쳐에 대해 확인하기에는 어려움이 있다고 사료된다.

#위를 통해 알 수 있는 문제점
데이터에서 얻을 수 있는 모든 거래정보(transactions) 에 대해, 1:1로 해당되는 identity 정보가 있지 않다.

# Target 값의 분포 확인하기
본 프로젝트값을 통해 예측해내야 하는 y값. Target 에 대해 몇 개의 플롯을 그려 확인해본다. 해당 데이터셋에서는 IsFraud와 동일하다.

In [ ]:
df_trans['TransactionAmt'] = df_trans['TransactionAmt'].astype(float)
total = len(df_trans)
total_amt = df_trans.groupby(['isFraud'])['TransactionAmt'].sum().sum()
plt.figure(figsize=(16,6))

plt.subplot(121)
g = sns.countplot(x='isFraud', data=df_trans, )
g.set_title("Fraud Transactions Distribution \n# 0: No Fraud | 1: Fraud #", fontsize=22)
g.set_xlabel("Is fraud?", fontsize=18)
g.set_ylabel('Count', fontsize=18)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=15) 

perc_amt = (df_trans.groupby(['isFraud'])['TransactionAmt'].sum())
perc_amt = perc_amt.reset_index()
plt.subplot(122)
g1 = sns.barplot(x='isFraud', y='TransactionAmt',  dodge=True, data=perc_amt)
g1.set_title("% Total Amount in Transaction Amt \n# 0: No Fraud | 1: Fraud #", fontsize=22)
g1.set_xlabel("Is fraud?", fontsize=18)
g1.set_ylabel('Total Transaction Amount Scalar', fontsize=18)
for p in g1.patches:
    height = p.get_height()
    g1.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total_amt * 100),
            ha="center", fontsize=15) 
    
plt.show()

# 가볍게 얻을 수 있는 인사이트

* 데이터셋 자체에서 불균형을 확인할 수 있다. 다시 말해, Isfraud 값이 압도적으로 적다.
* 전체 거래 분포도에서 데이터셋의 fraud 분포는 3.5%이다.
* 백분율의 금액이 총 거래량 금액의 3.5%보다 높은지 낮은지를 알아보는 것이 흥미로울 것 같다.
* % Total Transactions Amount의 fraud와 전체 거래에서의 fraud를 살펴보면, 비슷한 %를 가지고 있다는 것을 알 수 있다.
* 여러모로 거래량 항목에 대해 더 자세히 살펴볼 필요가 있다.

# 문제점 하나 더

우리가 제공받은 것은 Class imbalance한 데이터셋이다.
이 데이터 프레임을 예측 모델 및 분석의 기반으로 사용할 경우 많은 오류가 발생할 수 있으며, 대부분의 거래가 Fraud가 아니라고 예측할 확률이 높다. 

# Transaction Amount의 분위 확인
거래 금액을 Ploting 하기 전에 Transaction Amount의 분위수를 확인해보자.

In [ ]:
df_trans['TransactionAmt'] = df_trans['TransactionAmt'].astype(float)
print("Transaction Amounts Quantiles:")
print(df_trans['TransactionAmt'].quantile([.01, .025, .1, .25, .5, .75, .9, .975, .99]))

In [ ]:
df_trans['TransactionDT'].plot(kind='hist',
                                        figsize=(15, 5),
                                        label='train',
                                        bins=50,
                                        title='Train vs Test TransactionDT distribution')
test_transaction['TransactionDT'].plot(kind='hist',
                                       label='test',
                                       bins=50)
plt.legend()
plt.show()

train 과 test의 거래 일수 (transaction dates) 가 겹치지 않는 것 같으며, 이 데이터셋을 제대로 파악하고 validation 하기 위해서시간 기반으로 데이터셋을 분할해야 한다. 이 내용은 https://www.kaggle.com/robikscube/ieee-fraud-detection-first-look-and-eda 에서도 확인할 수 있다. 두 데이터셋 사이에는 대략 30일의 간격이 있음을 알 수 있다.

In [ ]:
del  train_identity,test_identity, test_transaction

# Transaction Amount의 value값 분포 확인

In [ ]:
plt.figure(figsize=(16,12))
plt.subplot(222)
g1 = sns.distplot(np.log(df_trans['TransactionAmt']))
g1.set_title("Transaction Amount (Log) Distribuition", fontsize=18)
g1.set_xlabel("")
g1.set_ylabel("Probability", fontsize=15)

plt.figure(figsize=(16,12))
plt.subplot(212)
g4 = plt.scatter(range(df_trans[df_trans['isFraud'] == 0].shape[0]),
                 np.sort(df_trans[df_trans['isFraud'] == 0]['TransactionAmt'].values), 
                 label='NoFraud', alpha=.2)
g4 = plt.scatter(range(df_trans[df_trans['isFraud'] == 1].shape[0]),
                 np.sort(df_trans[df_trans['isFraud'] == 1]['TransactionAmt'].values), 
                 label='Fraud', alpha=.2)
g4= plt.title("ECDF \nFRAUD and NO FRAUD Transaction Amount Distribution", fontsize=18)
g4 = plt.xlabel("Index")
g4 = plt.ylabel("Amount Distribution", fontsize=15)
g4 = plt.legend()

plt.figure(figsize=(16,12))

plt.subplot(321)
g = plt.scatter(range(df_trans[df_trans['isFraud'] == 1].shape[0]), 
                 np.sort(df_trans[df_trans['isFraud'] == 1]['TransactionAmt'].values), 
                label='isFraud', alpha=.4)
plt.title("FRAUD - Transaction Amount ECDF", fontsize=18)
plt.xlabel("Index")
plt.ylabel("Amount Distribution", fontsize=12)

plt.subplot(322)
g1 = plt.scatter(range(df_trans[df_trans['isFraud'] == 0].shape[0]),
                 np.sort(df_trans[df_trans['isFraud'] == 0]['TransactionAmt'].values), 
                 label='NoFraud', alpha=.2)
g1= plt.title("NO FRAUD - Transaction Amount ECDF", fontsize=18)
g1 = plt.xlabel("Index")
g1 = plt.ylabel("Amount Distribution", fontsize=15)

plt.suptitle('Individual ECDF Distribution', fontsize=22)

plt.show()

* Transaction Values Distribution 이라는 이름으로 distplot을 생성한다.
* Transaction Amount에 로그를 취해 본 분포와, 전체적인  FRAUD와 NO FRAUD의 분포 그래프를 그려 확인해본다.
* FRAUD의 거래 양 분포와 NO FRAUD의 거래 양 분포도 대략적으로 알 수 있었다.

아래의 3 개 그래프는 경험적 누적분포 함수 ECDF를 썼으며, 서로 다른 표본들의 분포를 비교할 때 많이 사용하고, 각 집단의 백분위를 추정할 수 있다는 장점을 가졌다.

# 위를 통해 얻어낼 수 있는 인사이트
* 거래량에 log를 취한 그래프의 모양이 굉장히 예쁘고 바람직하게 생겼다. 어쩌면, 대회 주최측에서는 어느 정도 가공된 데이터를 제공한 것일지도 모른다.
* Fraud, 즉 사기 금액의 액수 분포는 꽤 한정적이다. 최고한도가 3000을 조금 웃도는 수준에서 머물고, 가끔 이례적으로 5000까지를 찍고 있다.

# Fraud and No Fraud 의 분위수 보기

In [ ]:
print(pd.concat([df_trans[df_trans['isFraud'] == 1]['TransactionAmt']\
                 .quantile([.01, .1, .25, .5, .75, .9, .99])\
                 .reset_index(), 
                 df_trans[df_trans['isFraud'] == 0]['TransactionAmt']\
                 .quantile([.01, .1, .25, .5, .75, .9, .99])\
                 .reset_index()],
                axis=1, keys=['Fraud', "No Fraud"]))

# Transaction Amount의 아웃라이어
앞서 정의한 함수를 통해 위에서 확인한 분포값의 이상치, 다시 말해 아웃라이어 구간을 확인해보기로 한다.

참고로, 평균std이 표준값mean의 3배보다 높은 outlier values값을 가진다.

In [ ]:
CalcOutliers(df_trans['TransactionAmt'])

* Identified lowest outliers: 0
* Identified upper outliers: 10097
* 총 outlier 관측치: 10097
* Non-outlier 관측치 : 580443
* Outlier의 총 백분율:  1.7395

 >= 0 to 800  사이의 values들만 고려한다면, outlier를 피하고 분포의 신뢰도를 높일 수 있을 것이다.
 
 >=outlier인 10k row는 전체 행의 1.74%이다. (소수점 3째자리에서 반올림했을 때)

# Product Feature 알아보기

* Product 피쳐의 value값을  분포 확인
* Product의 Frauds 분포 알아보기
* Product의 Transaction 과 Amounts에 차이점이 있을까?

스포하자면, 사실 이 Feature은 그리 중요한 가중치를 가지진 않는다.

In [ ]:
tmp = pd.crosstab(df_trans['ProductCD'], df_trans['isFraud'], normalize='index') * 100
tmp = tmp.reset_index()
tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

plt.figure(figsize=(14,10))
plt.suptitle('ProductCD Distributions', fontsize=22)

plt.subplot(221)
g = sns.countplot(x='ProductCD', data=df_trans)
# plt.legend(title='Fraud', loc='upper center', labels=['No', 'Yes'])

g.set_title("ProductCD Distribution", fontsize=19)
g.set_xlabel("ProductCD Name", fontsize=17)
g.set_ylabel("Count", fontsize=17)
g.set_ylim(0,500000)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=14) 

plt.subplot(222)
g1 = sns.countplot(x='ProductCD', hue='isFraud', data=df_trans)
plt.legend(title='Fraud', loc='best', labels=['No', 'Yes'])
gt = g1.twinx()
gt = sns.pointplot(x='ProductCD', y='Fraud', data=tmp, color='black', order=['W', 'H',"C", "S", "R"], legend=False)
gt.set_ylabel("% of Fraud Transactions", fontsize=16)

g1.set_title("Product CD by Target(isFraud)", fontsize=19)
g1.set_xlabel("ProductCD Name", fontsize=17)
g1.set_ylabel("Count", fontsize=17)

plt.subplot(212)
g3 = sns.boxenplot(x='ProductCD', y='TransactionAmt', hue='isFraud', 
              data=df_trans[df_trans['TransactionAmt'] <= 2000] )
g3.set_title("Transaction Amount Distribuition by ProductCD and Target", fontsize=20)
g3.set_xlabel("ProductCD Name", fontsize=17)
g3.set_ylabel("Transaction Values", fontsize=17)

plt.subplots_adjust(hspace = 0.6, top = 0.85)

plt.show()

# 위를 통해 가볍게 알아낼 수 있는 것
* W, C , R 이 가장 빈도가 높은 값들이다. 그 중에서도 W가 혼자서 74.45%의 점유율을 가지며, 압도적인 데이터 분포를 가진다. 하지만 그렇다고 해서 Fraud 값이 다른 것들보다 높은 것도 아니다.
* W, H, R 의 Fraud값의 분포가 Non-Fraud 보다 약간 더 높다는 것을 알 수 있다.
* 사실상 ProductCD 마스킹이 되어 있기 때문에, 이렇게 눈으로 보는 것으로는 유의미한 인사이트를 도출하는 데에 한계가 있다.
* 다만 데이터의 분포를 파악해볼 수는 있었다.

# Card Features 알아보기

*  앞서 가장 먼저 데이터의 전체적인 모양을 파악할 때 알았듯, card features들은 categorical 피쳐들이다.
*  피쳐 값들의 분포를 알아보도록 하자.
* 또한 features의 각 밸류값에 대한 transactions과 '% of Fraud'를 알아보자.
* Card features에는 6개의 열이 있고, 그 중 4개는 numericals 피쳐인 것 같다. 따라서 이것들에 대해서는 분위수와 분포를 확인해보자.
* 다시 한 번 스포하자면, Card1과 Card2는 Fraud 예측에 높은 가중치를 가진다.

In [ ]:
##  Card Features 들 미리 살펴보기
resumetable(df_trans[['card1', 'card2', 'card3','card4', 'card5', 'card6']])

# 위를 통해 알아낸 것
* Card1에는 결측값이 하나도 없다. 
* Card2-Card6들은 결측값이 있다. 이것을 나중에 메워야 한다. 
* Card4와 Card6은 object type이다.

# Card 데이터셋 : Numericals Feature 들의 분위수

 object type인 Card4와 Card6은 제외하고 확인해보자.

In [ ]:
print("Card Features Quantiles: ")
print(df_trans[['card1', 'card2', 'card3', 'card5']].quantile([0.01, .025, .1, .25, .5, .75, .975, .99]))

Card 1 과 Card 2의 값들이 비교적으로  큰 분포도를 가지고 있다는 것을 볼 수 있다. 

그러므로 해당 columns들에 대해선 log값을 얻는 것이 더 나을 것이다.

In [ ]:
df_trans.loc[df_trans.card3.isin(df_trans.card3.value_counts()[df_trans.card3.value_counts() < 200].index), 'card3'] = "Others"
df_trans.loc[df_trans.card5.isin(df_trans.card5.value_counts()[df_trans.card5.value_counts() < 300].index), 'card5'] = "Others"

# Card 1, Card 2 Card 3의 분포 시각화하기

* Card 1 과 Card 2는 numerical 피쳐들이므로, 분포도를 그릴 수 있다.
* Card 3은 낮은 빈도를 가진 값이 많이 있기 때문에, 따로 나누어 "Others" value에 넣었다.
* 또한 Card 3에서 % of Fraud 값을 y axis2로 설정했다.

In [ ]:
# card3과 card5를  normalize하고 인덱스를 리셋한다. column명도 직관적으로 바꿔준다.
tmp = pd.crosstab(df_trans['card3'], df_trans['isFraud'], normalize='index') * 100
tmp = tmp.reset_index()
tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

tmp2 = pd.crosstab(df_trans['card5'], df_trans['isFraud'], normalize='index') * 100
tmp2 = tmp2.reset_index()
tmp2.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)


## 플롯을 그려본다 
## card1 card2 의 타겟 밸류값의 분포를 distplot로 출력

plt.figure(figsize=(14,22))

plt.subplot(411)
g = sns.distplot(df_trans[df_trans['isFraud'] == 1]['card1'], label='Fraud')
g = sns.distplot(df_trans[df_trans['isFraud'] == 0]['card1'], label='NoFraud')
g.legend()
g.set_title("Card 1 Values Distribution by Target", fontsize=20)
g.set_xlabel("Card 1 Values", fontsize=18)
g.set_ylabel("Probability", fontsize=18)

plt.subplot(412)
g1 = sns.distplot(df_trans[df_trans['isFraud'] == 1]['card2'].dropna(), label='Fraud')
g1 = sns.distplot(df_trans[df_trans['isFraud'] == 0]['card2'].dropna(), label='NoFraud')
g1.legend()
g1.set_title("Card 2 Values Distribution by Target", fontsize=20)
g1.set_xlabel("Card 2 Values", fontsize=18)
g1.set_ylabel("Probability", fontsize=18)


##  card3 밸류값의 분포와  Transaction Frauds 비율


plt.subplot(413)
g2 = sns.countplot(x='card3', data=df_trans, order=list(tmp.card3.values))
g22 = g2.twinx()
gg2 = sns.pointplot(x='card3', y='Fraud', data=tmp, 
                    color='black', order=list(tmp.card3.values))
gg2.set_ylabel("% of Fraud Transactions", fontsize=16)
g2.set_title("Card 3 Values Distribution and % of Transaction Frauds", fontsize=20)
g2.set_xlabel("Card 3 Values", fontsize=18)
g2.set_ylabel("Count", fontsize=18)
for p in g2.patches:
    height = p.get_height()
    g2.text(p.get_x()+p.get_width()/2.,
            height + 25,
            '{:1.2f}%'.format(height/total*100),
            ha="center") 

plt.subplot(414)
g3 = sns.countplot(x='card5', data=df_trans, order=list(tmp2.card5.values))
g3t = g3.twinx()
g3t = sns.pointplot(x='card5', y='Fraud', data=tmp2, 
                    color='black', order=list(tmp2.card5.values))
g3t.set_ylabel("% of Fraud Transactions", fontsize=16)
g3.set_title("Card 5 Values Distribution and % of Transaction Frauds", fontsize=20)
g3.set_xticklabels(g3.get_xticklabels(),rotation=90)
g3.set_xlabel("Card 5 Values", fontsize=18)
g3.set_ylabel("Count", fontsize=18)
for p in g3.patches:
    height = p.get_height()
    g3.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center",fontsize=11) 
    
plt.subplots_adjust(hspace = 0.6, top = 0.85)


plt.show()

# 위를 통해 알아낼 수 있는 인사이트

card 1에서는 Fraud와 not Fraud의 분포 비율이 상당히 비슷하다는 사실을 알 수 있다. 사기에 상대적으로 취약한 카드인 걸지도 모른다. 이것만 단편적으로 보아도, target값 산출에 상당히 주요한 역할을 할 것임을 짐작할 수 있다.


card 2에서는 상대적으로 not Fraud의 분포 비율이 높다. 하지만,  Fraud 비율도 무시할 수 없을 정도로 확인된다. 역시 target값 산출에 상당히 주요한 역할을 할 것임을 짐작할 수 있다.


card 3에서는 150.0의 value값에 88.27%의 거래량이 몰려 있는 것을 알 수 있다. 하지만 Fraud Transaction의 퍼센트%는 상대적으로 낮음을 알 수 있다. 대략 3%를 조금 웃도는 수준이다. 반대로, 158.0  value값에는 남은 9.54%의 거래량이 몰려 있는데,  Fraud Transaction의 퍼센트%가 매우 높은 것을 확인할 수 있다. 12%를 웃도는 수준이다. 거래량이 거의 없는 119, 144, 185항목이 높은 Frauds 비율을 보인다.


card 5에서는 226.0의 value값에 50.2%의 거래량이 몰려 있다. 하지만 총 거래량 대비 사기율은 낮은 편이다. 오히려 거래량이 적은 137.0에서 가장 높은 Fraud Transaction 퍼센트를 파악할 수 있다. card5 에서 가장 빈도가 높은 values는 데이터의 73%를 차지하는 226, 224, 166이다. values값이 거의 없는 137, 141, 147, 223 항목이 높은 Frauds 비율을 보인다.

# Card 4 :  Categorical Feature

In [ ]:
# card4를 정규화해서 분포를 확인한다.

tmp = pd.crosstab(df_trans['card4'], df_trans['isFraud'], normalize='index') * 100
tmp = tmp.reset_index()
tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)



plt.figure(figsize=(14,10))
plt.suptitle('Card 4 Distributions', fontsize=22)

plt.subplot(221)
g = sns.countplot(x='card4', data=df_trans)
# plt.legend(title='Fraud', loc='upper center', labels=['No', 'Yes'])
g.set_title("Card4 Distribution", fontsize=19)
g.set_ylim(0,420000)
g.set_xlabel("Card4 Category Names", fontsize=17)
g.set_ylabel("Count", fontsize=17)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center",fontsize=14) 


plt.subplot(222)
g1 = sns.countplot(x='card4', hue='isFraud', data=df_trans)
plt.legend(title='Fraud', loc='best', labels=['No', 'Yes'])
gt = g1.twinx()
gt = sns.pointplot(x='card4', y='Fraud', data=tmp, 
                   color='black', legend=False, 
                   order=['discover', 'mastercard', 'visa', 'american express'])
gt.set_ylabel("% of Fraud Transactions", fontsize=16)
g1.set_title("Card4 by Target(isFraud)", fontsize=19)
g1.set_xlabel("Card4 Category Names", fontsize=17)
g1.set_ylabel("Count", fontsize=17)

plt.subplot(212)
g3 = sns.boxenplot(x='card4', y='TransactionAmt', hue='isFraud', 
              data=df_trans[df_trans['TransactionAmt'] <= 2000] )
g3.set_title("Card 4 Distribuition by ProductCD and Target", fontsize=20)
g3.set_xlabel("Card4 Category Names", fontsize=17)
g3.set_ylabel("Transaction Values", fontsize=17)

plt.subplots_adjust(hspace = 0.6, top = 0.85)

plt.show()

# 위를 통해 알 수 있는 인사이트

card 4에 대한 항목을 살펴보면, visa 카드와 mastercard가 대부분의 거래량을 차지하고 있는 것을 알 수 있다. 각각 65.16%과 32.04%의 점유율을 가진다.


타겟값을 가지고 비교를 해 보았을 때, 안정적으로 적은 비율로 거래 사기 Fraud가 발생하고 있음을 알 수 있다. visa 카드가 3%가 덜 되는 사기 거래 퍼센테이지를, master카드는 그것보다 조금 더 적은 퍼센테이지를 가진다.


한편 discover 의  Fraud value값은 분포가 높은 것을 확인할 수 있다. 나머지 카드들은 정상거래와 사기거래가 비슷한 분포도를 가지고 있다.
discover의 is Fraud value 값이 가장 높고 (8%) Mastercard와 Visa (3.5%) American Express( 2.87% ) 순이다.

# Card 6 - Categorical Feature

In [ ]:
tmp = pd.crosstab(df_trans['card6'], df_trans['isFraud'], normalize='index') * 100
tmp = tmp.reset_index()
tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

plt.figure(figsize=(14,10))
plt.suptitle('Card 6 Distributions', fontsize=22)

plt.subplot(221)
g = sns.countplot(x='card6', data=df_trans, order=list(tmp.card6.values))
# plt.legend(title='Fraud', loc='upper center', labels=['No', 'Yes'])
g.set_title("Card6 Distribution", fontsize=19)
g.set_ylim(0,480000)
g.set_xlabel("Card6 Category Names", fontsize=17)
g.set_ylabel("Count", fontsize=17)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center",fontsize=14) 

plt.subplot(222)
g1 = sns.countplot(x='card6', hue='isFraud', data=df_trans, order=list(tmp.card6.values))
plt.legend(title='Fraud', loc='best', labels=['No', 'Yes'])
gt = g1.twinx()
gt = sns.pointplot(x='card6', y='Fraud', data=tmp, order=list(tmp.card6.values),
                   color='black', legend=False, )
gt.set_ylim(0,20)
gt.set_ylabel("% of Fraud Transactions", fontsize=16)
g1.set_title("Card6 by Target(isFraud)", fontsize=19)
g1.set_xlabel("Card6 Category Names", fontsize=17)
g1.set_ylabel("Count", fontsize=17)

plt.subplot(212)
g3 = sns.boxenplot(x='card6', y='TransactionAmt', hue='isFraud', order=list(tmp.card6.values),
              data=df_trans[df_trans['TransactionAmt'] <= 2000] )
g3.set_title("Card 6 Distribuition by ProductCD and Target", fontsize=20)
g3.set_xlabel("Card6 Category Names", fontsize=17)
g3.set_ylabel("Transaction Values", fontsize=17)

plt.subplots_adjust(hspace = 0.6, top = 0.85)

plt.show()

# 위를 통해 얻을 수 있는 인사이트

card 4에 대한 항목을 살펴보면, 신용Credit 거래  에 해당하는 거래량이 74.5% 압도적으로 높은 것을 확인할 수 있다.

인출debit 거래의 경우, 25.23%로 나머지 비율을 거의 다 가져간다.

하지만 반대로 Fraud 거래 비율은 인출debit 거래가 Credit 신용거래보다 낮음을 알 수 있다. 

Transaction Amount 분포를 보면, value들 간에 명확한 차이가 나타나지 않는다.


# Exploring M1-M9 Features
M1-M9 : match, such as names on card and address, etc.

In [ ]:
for col in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']:
    df_trans[col] = df_trans[col].fillna("Miss")
    
def ploting_dist_ratio(df, col, lim=2000):
    tmp = pd.crosstab(df[col], df['isFraud'], normalize='index') * 100
    tmp = tmp.reset_index()
    tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

    plt.figure(figsize=(20,5))
    plt.suptitle(f'{col} Distributions ', fontsize=22)

    plt.subplot(121)
    g = sns.countplot(x=col, data=df, order=list(tmp[col].values))
    # plt.legend(title='Fraud', loc='upper center', labels=['No', 'Yes'])
    g.set_title(f"{col} Distribution\nCound and %Fraud by each category", fontsize=18)
    g.set_ylim(0,400000)
    gt = g.twinx()
    gt = sns.pointplot(x=col, y='Fraud', data=tmp, order=list(tmp[col].values),
                       color='black', legend=False, )
    gt.set_ylim(0,20)
    gt.set_ylabel("% of Fraud Transactions", fontsize=16)
    g.set_xlabel(f"{col} Category Names", fontsize=16)
    g.set_ylabel("Count", fontsize=17)
    for p in gt.patches:
        height = p.get_height()
        gt.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center",fontsize=14) 
        
    perc_amt = (df_trans.groupby(['isFraud',col])['TransactionAmt'].sum() / total_amt * 100).unstack('isFraud')
    perc_amt = perc_amt.reset_index()
    perc_amt.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

    plt.subplot(122)
    g1 = sns.boxplot(x=col, y='TransactionAmt', hue='isFraud', 
                     data=df[df['TransactionAmt'] <= lim], order=list(tmp[col].values))
    g1t = g1.twinx()
    g1t = sns.pointplot(x=col, y='Fraud', data=perc_amt, order=list(tmp[col].values),
                       color='black', legend=False, )
    g1t.set_ylim(0,5)
    g1t.set_ylabel("%Fraud Total Amount", fontsize=16)
    g1.set_title(f"{col} by Transactions dist", fontsize=18)
    g1.set_xlabel(f"{col} Category Names", fontsize=16)
    g1.set_ylabel("Transaction Amount(U$)", fontsize=16)
        
    plt.subplots_adjust(hspace=.4, wspace = 0.35, top = 0.80)
    
    plt.show()

## M 분포 확인하기 : Count, %Fraud and Transaction Amount distribution

C Feature처럼 마스킹되어 있는 항목들이다. 다시금 스포하자면, 이 항목 또한 taget값 예측에 중요한 가중치를 가지진 않는다.

In [ ]:
for col in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']:
    ploting_dist_ratio(df_trans, col, lim=2500)

# Addr1 and Addr2
addr: address 

카드 사용자들의 주소지 정보가 담긴 Feature이다.

In [ ]:
#분위수를 살펴본다.
print("Card Features Quantiles: ")
print(df_trans[['addr1', 'addr2']].quantile([0.01, .025, .1, .25, .5, .75, .90,.975, .99]))

* Addr1에서 5000개 미만의 입력 항목을 가진 모든 values값을 "Others"로 설정한다. 한편, 전체적으로도 Addr2보다 훨씬 많은 value값을 가지고 있음을 알 수 있다.  
* Addr2에서 50개 미만의 모든 values값을 "Others"로 설정한다.

In [ ]:
df_trans.loc[df_trans.addr1.isin(df_trans.addr1.value_counts()[df_trans.addr1.value_counts() <= 5000 ].index), 'addr1'] = "Others"
df_trans.loc[df_trans.addr2.isin(df_trans.addr2.value_counts()[df_trans.addr2.value_counts() <= 50 ].index), 'addr2'] = "Others"

# Addr1 분포 확인하기

In [ ]:
def ploting_cnt_amt(df, col, lim=2000):
    tmp = pd.crosstab(df[col], df['isFraud'], normalize='index') * 100
    tmp = tmp.reset_index()
    tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)
    
    plt.figure(figsize=(16,14))    
    plt.suptitle(f'{col} Distributions ', fontsize=24)
    
    plt.subplot(211)
    g = sns.countplot( x=col,  data=df, order=list(tmp[col].values))
    gt = g.twinx()
    gt = sns.pointplot(x=col, y='Fraud', data=tmp, order=list(tmp[col].values),
                       color='black', legend=False, )
    gt.set_ylim(0,tmp['Fraud'].max()*1.1)
    gt.set_ylabel("%Fraud Transactions", fontsize=16)
    g.set_title(f"Most Frequent {col} values and % Fraud Transactions", fontsize=20)
    g.set_xlabel(f"{col} Category Names", fontsize=16)
    g.set_ylabel("Count", fontsize=17)
    g.set_xticklabels(g.get_xticklabels(),rotation=45)
    sizes = []
    for p in g.patches:
        height = p.get_height()
        sizes.append(height)
        g.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center",fontsize=12) 
        
    g.set_ylim(0,max(sizes)*1.15)
    
    #########################################################################
    perc_amt = (df.groupby(['isFraud',col])['TransactionAmt'].sum() \
                / df.groupby([col])['TransactionAmt'].sum() * 100).unstack('isFraud')
    perc_amt = perc_amt.reset_index()
    perc_amt.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)
    amt = df.groupby([col])['TransactionAmt'].sum().reset_index()
    perc_amt = perc_amt.fillna(0)
    plt.subplot(212)
    g1 = sns.barplot(x=col, y='TransactionAmt', 
                       data=amt, 
                       order=list(tmp[col].values))
    g1t = g1.twinx()
    g1t = sns.pointplot(x=col, y='Fraud', data=perc_amt, 
                        order=list(tmp[col].values),
                       color='black', legend=False, )
    g1t.set_ylim(0,perc_amt['Fraud'].max()*1.1)
    g1t.set_ylabel("%Fraud Total Amount", fontsize=16)
    g.set_xticklabels(g.get_xticklabels(),rotation=45)
    g1.set_title(f"{col} by Transactions Total + %of total and %Fraud Transactions", fontsize=20)
    g1.set_xlabel(f"{col} Category Names", fontsize=16)
    g1.set_ylabel("Transaction Total Amount(U$)", fontsize=16)
    g1.set_xticklabels(g.get_xticklabels(),rotation=45)    
    
    for p in g1.patches:
        height = p.get_height()
        g1.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total_amt*100),
                ha="center",fontsize=12) 
        
    plt.subplots_adjust(hspace=.4, top = 0.9)
    plt.show()
    
ploting_cnt_amt(df_trans, 'addr1')

# 위에서 얻을 수 있는 인사이트

addr1 항목에서, 흥미로운 사실들을 발견할 수 있다. 주소지의 카테고리 이름에 따라 사기거래 확률의 퍼센트가 확연히 달라진다는 점이다.

첫 번째 그래프는 가장 빈번하게 확인되는 addr1의 value값과 사기거래 Fraud의 퍼센트를 나타내고 있다. 269, others, 330, 476에서 Fraud Transactions의 %가 특히 높게 두드러진다.

두 번째 그래프는 총 거래량과 총 거래량의 %, Fraud Transactions의 %를 확인할 수 있다. 이 그래프에서는 조금 더 명확하게 value별 Fraud Transactions의 %를 확인할 수 있다. 251, 191, others, 472, 512에서 특히 높은 사기 거래 비율을 확인할 수 있다.

이 Feature은 분명히 유의미한 가중치를 가질 것임을 예측할 수 있다.

# Addr2 분포 확인

In [ ]:
ploting_cnt_amt(df_trans, 'addr2')

# 알 수 있는 인사이트

첫 번째 그래프는 가장 빈번하게 확인되는 addr1의 value값과 사기거래 Fraud의 퍼센트를 나타내고 있다.
Addr2의 거의 모든 항목이 0.01퍼센트대의 비슷한 value값을 가진다. 87 value는 전체 항목의 88.14% 를 차지하며, 나머지는 어디로 갔는지 모르겠다...
재미있는 점은, 65 value에서 frauds 의 비율은 거의 60% 에 육박한다는 점이다.

두 번째 그래프는 총 거래량과 총 거래량의 %, Fraud Transactions의 %를 확인할 수 있다. 87 value는 총 Transaction Amounts 의 96%를 차지한다. 여전히 65 value에서 frauds 의 비율은 거의 60% 에 육박한다. 

우리는 이 Feature 또한 중요한 가중치를 가지게 될 것임을 알 수 있다.

# P emaildomain 분포 확인
purchaser and recipient email domain

모든 전자 메일 도메인을 각 기업별로 그룹화하고, 500개 미만의 값을 모두 "Others"로 분류한다.

In [ ]:
df_trans.loc[df_trans['P_emaildomain'].isin(['gmail.com', 'gmail']),'P_emaildomain'] = 'Google'

df_trans.loc[df_trans['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                         'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                         'yahoo.es']), 'P_emaildomain'] = 'Yahoo Mail'
df_trans.loc[df_trans['P_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                         'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                         'outlook.es', 'live.com', 'live.fr',
                                         'hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
df_trans.loc[df_trans.P_emaildomain.isin(df_trans.P_emaildomain\
                                         .value_counts()[df_trans.P_emaildomain.value_counts() <= 500 ]\
                                         .index), 'P_emaildomain'] = "Others"
df_trans.P_emaildomain.fillna("NoInf", inplace=True)

# P-Email Domain 플롯팅 그려보기

In [ ]:
ploting_cnt_amt(df_trans, 'P_emaildomain')

# R-Email Domain plot 분포 확인

* 모든 전자 메일 도메인을 각 기업별로 그룹화한다.
* 300개 미만의 값을 모두 "Others"로 설정한다.

In [ ]:
df_trans.loc[df_trans['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'

df_trans.loc[df_trans['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                             'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                             'yahoo.es']), 'R_emaildomain'] = 'Yahoo Mail'
df_trans.loc[df_trans['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                             'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr',
                                             'hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
df_trans.loc[df_trans.R_emaildomain.isin(df_trans.R_emaildomain\
                                         .value_counts()[df_trans.R_emaildomain.value_counts() <= 300 ]\
                                         .index), 'R_emaildomain'] = "Others"
df_trans.R_emaildomain.fillna("NoInf", inplace=True)

In [ ]:
ploting_cnt_amt(df_trans, 'R_emaildomain')

# 얻을 수 있는 인사이트

* email domain features들은 매우 유사한 분포도를 가지고 있다.
* google과  icloud frauds 의 frauds가 높다는 것을 확인할 수 있다.

# C1-C14 features 분포 확인하기

C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.

실제 의미는 마스킹된, 카드에 연관된 주소 등의 정보이다.

In [ ]:
resumetable(df_trans[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
                      'C9', 'C10', 'C11', 'C12', 'C13', 'C14']])

전부 Datatype이 float16 임을 알 수 있다. 결측치는 없고, Value값은 0.0~2.0 사이로 보인다.

In [ ]:
df_trans[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
                      'C9', 'C10', 'C11', 'C12', 'C13', 'C14']].describe()

In [ ]:
df_trans.loc[df_trans.C1.isin(df_trans.C1\
                              .value_counts()[df_trans.C1.value_counts() <= 400 ]\
                              .index), 'C1'] = "Others"

# TimeDelta Feature

frauds가 발생하는 비율이 높은 특정 시간이 있는지 확인해 보자.

## 총 요일, 주중 및 시간으로 변환

첫 번째 날짜를 2017-12-01로 사용하고  delta time 을 사용하여 날짜/시간 기능을 계산한다.

In [ ]:
# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100400#latest-579480
import datetime

START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
df_trans["Date"] = df_trans['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))

df_trans['_Weekdays'] = df_trans['Date'].dt.dayofweek
df_trans['_Hours'] = df_trans['Date'].dt.hour
df_trans['_Days'] = df_trans['Date'].dt.day

## Total Transaction Amount이 가장 높은 상위 일수

In [ ]:
ploting_cnt_amt(df_trans, '_Days')

# 얻어갈 수 있는 인사이트
전체적으로 고른 분포를 가지고 있지만, 특히 월초와 월에 FRaud Total Amount가 높은 것을 알 수 있다.

##  WeekDays 분포도

In [ ]:
ploting_cnt_amt(df_trans, '_Weekdays')

# 얻을 수 있는 인사이트
날짜의 정보는 없지만 2일 가량의 거래량이 적다는 것을 알 수 있다. 주말임을 추론할 수 있다.

반대로 오히려 Fraud 사기 거래의 총계는 주말에 높다는 것을 알 수 있다.

## 시간 분포도

In [ ]:
ploting_cnt_amt(df_trans, '_Hours')

# 얻을 수 있는 인사이트

아침 5, 6,7,8,9,10시에 거래 빈도수는 적지만, 그에 비해 Fraud 사기거래 총계 비율은 상당히 높은 것을 알 수 있다. 

거래 총량 + %총량 + %사기총량 그래프를 봐도 6~10시 사이에 빈도수 대비 Fraud 비율이 높은 것을 알 수 있다.

# 모델링 함수 선언

In [ ]:

@jit
def fast_auc(y_true, y_prob):
    """
    fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc


def eval_auc(y_true, y_pred):
    """
    Fast auc eval function for lgb.
    """
    return 'auc', fast_auc(y_true, y_pred), True


def train_model_classification(X, X_test, y, params, folds, model_type='lgb', eval_metric='auc', columns=None, plot_feature_importance=False, model=None,
                               verbose=10000, early_stopping_rounds=200, n_estimators=50000, splits=None, n_folds=3, averaging='usual', n_jobs=-1):

    columns = X.columns if columns is None else columns
    n_splits = folds.n_splits if splits is None else n_folds
    X_test = X_test[columns]
    
    # to set up scoring parameters
    metrics_dict = {'auc': {'lgb_metric_name': eval_auc,
                        'catboost_metric_name': 'AUC',
                        'sklearn_scoring_function': metrics.roc_auc_score},
                    }
    
    result_dict = {}
    if averaging == 'usual':
        # out-of-fold predictions on train data
        oof = np.zeros((len(X), 1))

        # averaged predictions on train data
        prediction = np.zeros((len(X_test), 1))
        
    elif averaging == 'rank':
        # out-of-fold predictions on train data
        oof = np.zeros((len(X), 1))

        # averaged predictions on train data
        prediction = np.zeros((len(X_test), 1))

    
    # list of scores on folds
    scores = []
    feature_importance = pd.DataFrame()
    
    # split and train on folds
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[columns][train_index], X[columns][valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            
        if model_type == 'lgb':
            model = lgb.LGBMClassifier(**params, n_estimators=n_estimators, n_jobs = n_jobs)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                    verbose=verbose, early_stopping_rounds=early_stopping_rounds)
            
            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=n_estimators, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=verbose, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            print('')
            
            y_pred = model.predict_proba(X_test)
        
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=n_estimators, eval_metric=metrics_dict[eval_metric]['catboost_metric_name'], **params,
                                      loss_function=Logloss)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        if averaging == 'usual':
            
            oof[valid_index] = y_pred_valid.reshape(-1, 1)
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
            
            prediction += y_pred.reshape(-1, 1)

        elif averaging == 'rank':
                                  
            oof[valid_index] = y_pred_valid.reshape(-1, 1)
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
                                  
            prediction += pd.Series(y_pred).rank().values.reshape(-1, 1)        
        
        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_splits
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['scores'] = scores
    
    if model_type == 'lgb':
        if plot_feature_importance:
            feature_importance["importance"] /= n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
            
            result_dict['feature_importance'] = feature_importance
            result_dict['top_columns'] = cols
        
    return result_dict



함수설명 
- train_model_classification() :파라미터를 입력받아 모델링을 실행하는 함수   
  lgb, xgb, sklearn, cat, usual, rank 를 고를수 있다.


# Feature engineering
집계를 생성해 보겠습니다. 그 안에는 논리가 없습니다. 단순히 상위 기능에 대한 집계입니다.

In [ ]:
for i in range(39):
    if(i<10 or i == 0):
        test.rename(columns={f'id-0{i}':f'id_0{i}'},inplace=True)
    else:
        test.rename(columns={f'id-{i}':f'id_{i}'},inplace=True)

In [ ]:
train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

train['id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
train['id_02_to_mean_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('mean')
train['id_02_to_std_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('std')
train['id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')

test['id_02_to_mean_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')
test['id_02_to_mean_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('mean')
test['id_02_to_std_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('std')
test['id_02_to_std_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('std')

train['D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
train['D15_to_mean_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('mean')
train['D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
train['D15_to_std_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('std')

test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
test['D15_to_mean_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('mean')
test['D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')
test['D15_to_std_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('std')

In [ ]:
train[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = train['P_emaildomain'].str.split('.', expand=True)
train[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = train['R_emaildomain'].str.split('.', expand=True)
test[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = test['P_emaildomain'].str.split('.', expand=True)
test[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = test['R_emaildomain'].str.split('.', expand=True)

# Prepare data for modelling

In [ ]:
many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.9]
many_null_cols_test = [col for col in test.columns if test[col].isnull().sum() / test.shape[0] > 0.9]

In [ ]:
big_top_value_cols = [col for col in train.columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in test.columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [ ]:
# nunique() 데이터 고유값의 총 수를 알고 싶을때 유용한 함수
one_value_cols = [col for col in train.columns if train[col].nunique() <=1]
one_value_cols_test = [col for col in test.columns if test[col].nunique()<=1]
one_value_cols == one_value_cols_test

In [ ]:
cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test + one_value_cols+ one_value_cols_test))
cols_to_drop.remove('isFraud')
len(cols_to_drop)

In [ ]:
train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [ ]:
cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']
for col in cat_cols:
    if col in train.columns:
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))   

In [ ]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)
y = train.sort_values('TransactionDT')['isFraud']
#X_test = test.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], axis=1)
X_test = test.drop(['TransactionDT', 'TransactionID'], axis=1)
del train
test = test[["TransactionDT", 'TransactionID']]

In [ ]:
# by https://www.kaggle.com/dimartinot
def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)   

# Cleaning infinite values to NaN
X = clean_inf_nan(X)
X_test = clean_inf_nan(X_test )

In [ ]:
gc.collect()


# LGBM

In [ ]:
n_fold = 5
folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=5)

In [ ]:
params = {'num_leaves': 256,
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': 13,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9,
          #'categorical_feature': cat_cols
         }
result_dict_lgb = train_model_classification(X=X, X_test=X_test, y=y, params=params, folds=folds, model_type='lgb', eval_metric='auc', plot_feature_importance=True,
                                                      verbose=500, early_stopping_rounds=200, n_estimators=5000, averaging='usual', n_jobs=-1)

In [ ]:
result_dict_lgb['prediction']

In [ ]:
sub['isFraud'] = result_dict_lgb['prediction']
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.head()


In [ ]:
pd.DataFrame(result_dict_lgb['oof']).to_csv('lgb_oof.csv', index=False)